In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/churn-modelling/Churn_Modelling.csv")
df

#  Removing unimport columns

In [ ]:
df=df.iloc[:,3:]
df.head()


In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()


# Using Heat map to check the null values in dataset
This heat map shows that there is no null value. The dataframe is already clean 

In [ ]:
sns.heatmap(df.isnull(),cmap="viridis")


# Now i'm using histogram to check the flow of data and also check the standard deviation of the data

In [ ]:
fig , ax =plt.subplots(2,2)
ax = ax.flatten()
fig.set_size_inches(14, 10)
sns.distplot(df.EstimatedSalary,color='#2D008E',ax=ax[0])
sns.distplot(df.CreditScore,color='#F9971A',ax=ax[1])
sns.distplot(df.Balance,color='#242852',ax=ax[2])
sns.distplot(df.Age,color='#242852',ax=ax[3])



# Remove outlier by using standard deviation
 
 ****upper limit and Lower limit of standard deviation in CreditScore 


In [ ]:
upper_limit_CreditScore= df["CreditScore"].mean() + 1.5*df["CreditScore"].std()
lower_limit_CreditScore = df["CreditScore"].mean() - 1.5*df["CreditScore"].std()
upper_limit_CreditScore , lower_limit_CreditScore

**Upper limit and Lower limit standard deviation of EstimatedSalary****

In [ ]:
upper_limit_EstimatedSalary= df["EstimatedSalary"].mean() + 1*df["EstimatedSalary"].std()
lower_limit_EstimatedSalary= df["EstimatedSalary"].mean() - 1.5*df["EstimatedSalary"].std()
upper_limit_EstimatedSalary , lower_limit_EstimatedSalary

# Now usin upper and lower limits of stds. i am removing outliers and create new data frame.****

In [ ]:
df1=df[(df["CreditScore"]<upper_limit_CreditScore)&(df["CreditScore"]>lower_limit_CreditScore)]
df2=df1[(df1["EstimatedSalary"]<upper_limit_EstimatedSalary)&(df1["EstimatedSalary"]>lower_limit_EstimatedSalary)]
df1.shape,df2.shape

# Now again using histogram
to check the flow of new data frame df2

In [ ]:
fig_1 , ax_1 =plt.subplots(1,2)
ax_1 = ax_1.flatten()
fig_1.set_size_inches(14, 6)
sns.distplot(df2.EstimatedSalary,color='#2D008E',ax=ax_1[0])
sns.distplot(df2.CreditScore,color='#F9971A',ax=ax_1[1])

In [ ]:
fig_3 , ax_3 =plt.subplots(2,2)
ax_3 = ax_3.flatten()
fig_3.set_size_inches(16, 8)
sns.violinplot(y=df2.CreditScore,x=df2.Tenure,palette="mako",ax=ax_3[0])
sns.barplot(y="CreditScore",x="Tenure",data=df2,palette="mako",ci="sd",ax=ax_3[1])
sns.violinplot(y=df2.CreditScore,x=df2.NumOfProducts,palette="rocket",ax=ax_3[2]) 
sns.barplot(y="CreditScore",x="NumOfProducts",data=df2,palette="rocket",ci="sd",ax=ax_3[3])


seperate churn 0 values and 1 to know the ratio

In [ ]:
df2_churn_1=df2[df2["Exited"]==1]
df2_churn_0=df2[df2["Exited"]==0]
df2_churn_1.shape,df2_churn_0.shape

# Labeling the gender column by 0 and 1

In [ ]:
df3 = df2.convert_dtypes()
df3["Gender"] = np.where(df3["Gender"]=='Female',0,1)

# creating a x dataset and y labels to train a model

In [ ]:
x=df3.drop(["Exited","Geography"],axis=1)
y=df3["Exited"]
y=y.astype('int')
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
x_train.shape,x_test.shape,y_train.shape

# Standarized the train and test data to increase the accuracy

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)


# Apply Logistic Regression model

In [ ]:
from sklearn.linear_model import  LogisticRegression
lr = LogisticRegression()
model_logic = lr.fit(x_train,y_train)

# Apply Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
model_rfc = rfc.fit(x_train,y_train)


# Apply Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy',max_depth=300)
model_dtc = dtc.fit(x_train,y_train)

# Apply support vector Machine

In [ ]:
from sklearn.svm import SVC
svc = SVC()
model_svc = svc.fit(x_train,y_train)


# Check the accuracy of all models by using Cross Validation Classifier

In [ ]:
from sklearn.model_selection import cross_val_score

scores_logic = cross_val_score(model_logic, x, y, cv=4)
scores_rfc = cross_val_score(model_rfc, x, y, cv=4)
scores_dtc = cross_val_score(model_dtc, x, y, cv=4)
scores_svc = cross_val_score(model_svc, x, y, cv=4)
data={'model':['logistic','random_forest','decision_tree','SVM'],
     'accuracy':[scores_logic,scores_rfc,scores_dtc,scores_svc]}
accuracy_df = pd.DataFrame(data, columns = ['model', 'accuracy'])
accuracy_df

In [ ]:
accuracy_df.accuracy

 #the accuracy results of show that random forest gain highest percentage which is 85%. 

# Using Confusion matrics to check the performance of random forest model

In [ ]:
y_predict = model_rfc.predict(x_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_predict)
cm


# Using heat map to plot confusion matrix. so that we can better understand the results

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cm,annot=True)
plt.xlabel('Predict')
plt.ylabel('Truth')


**#This heat map shows that the model predict  1159 times 0 and the actual value is also 0******

# Now i'm chect manually the result of my model 


In [ ]:
credit_score = 608
gender = 1 # which is male
age = 55
tenure = 6
balance = 83000
num_of_buy_product=12
has_a_creditcard = 1
is_active_member=0
salary=42000


arr=np.array([credit_score,gender,age,tenure,balance,num_of_buy_product,has_a_creditcard,is_active_member,salary])
arr_res=arr.reshape(1,-1)
a=model_rfc.predict(arr_res)
a